# Decoding Milkshake vs. H2O  
## Example code explanation  
Reference: https://nilearn.github.io/auto_examples/02_decoding/plot_haxby_anova_svm.html#sphx-glr-auto-examples-02-decoding-plot-haxby-anova-svm-py

In [ ]:
import os
import numpy as np
import nilearn
import glob
import nibabel as nib
import pandas as pd 
from sklearn.model_selection import cross_val_score
from nilearn.input_data import NiftiMasker 
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectKBest, f_classif
import numpy as np
np.seterr(divide='ignore', invalid='ignore')
import warnings
warnings.filterwarnings("ignore")
import matplotlib
matplotlib.use('Agg')

### Get the data paths 

In [ ]:
#image mask
imag_mask='/projects/niblab/bids_projects/Experiments/ChocoData/images/bin_mask.nii.gz'

#our behavioral csv file 
stim = '/projects/niblab/bids_projects/Experiments/ChocoData/behavorial_data/4w_smallset.csv'

#our dataset concatenated image 
dataset='/projects/niblab/bids_projects/Experiments/ChocoData/images/4w_smallset.nii.gz'

#load behavioral data into a pandas df
behavioral = pd.read_csv(stim, sep="\t")


### Load data

In [ ]:

# look at original unique labels 
print(behavioral["Label"].unique())


#grab conditional labels and set up milkshake
behavioral["Label"] = behavioral.replace(['HF_LS_receipt', 'LF_LS_receipt', 'LF_HS_receipt', 'HF_HS_receipt'], 'milkshake')


y = behavioral["Label"]
print(y.unique()) # make sure all the milkshake receipts have been replaced with "milkshake"


#restrict data to our target analysis 
condition_mask = behavioral["Label"].isin(['milkshake', "h20_receipt"])
y = y[condition_mask]


#confirm we have the # of condtions needed
print(y.unique())


masker = NiftiMasker(mask_img=imag_mask, standardize=True, memory="nilearn_cache", memory_level=1)
X = masker.fit_transform(dataset)
# Apply our condition_mask
X = X[condition_mask]




### Set up the prediction function -- a support vector classifier with a linear kernel

In [ ]:
# PREDICTION FUNCTION
from sklearn.svm import SVC
svc = SVC(kernel='linear')

### Set up our feature selection - ANOVA with "SelectKBest", k=500

In [ ]:
# FEATURE SELECTION
feature_selection = SelectKBest(f_classif, k=500)

### Put the prediction function and the classifier into a pipeline 

In [ ]:
anova_svc = Pipeline([('anova', feature_selection), ('svc', svc)])

### Fit pipeline 

In [ ]:
anova_svc.fit(X,y)
y_pred = anova_svc.predict(X)

### Run nested cross validation, cv=5

In [ ]:
# NESTED CROSS VALIDATION
from sklearn.model_selection import GridSearchCV
k_range = [ 15, 50, 150, 500, 1000, 3000, 5000]
grid = GridSearchCV(anova_svc, param_grid={'anova__k': k_range}, verbose=1, cv=5, n_jobs=4)
nested_cv_scores = cross_val_score(grid, X, y, cv=5)
#cv_scores = cross_val_score(anova_svc, X, conditions,)
# Print the results
print("Nested CV score: %.4f" % np.mean(nested_cv_scores))

### Save results (SVM weights) to an image (.png & .nii)

In [ ]:
# Here is the image 
coef = anov_svc.coef_
# reverse feature selection
coef = feature_selection.inverse_transform(coef)
# reverse masking
weight_img = masker.inverse_transform(coef)


# Use the mean image as a background to avoid relying on anatomical data
from nilearn import image
mean_img = image.mean_img(dataset)
mean_img.to_filename('/projects/niblab/bids_projects/Experiments/ChocoData/derivatives/code/decoding/milkshake_vs_h2O/images/4w_smallset_mean_nimask.nii')

# Create the figure
from nilearn.plotting import plot_stat_map, show
display = plot_stat_map(weight_img, mean_img, title='Milkshake vs. h2O , 4 waves')
display.savefig('/projects/niblab/bids_projects/Experiments/ChocoData/derivatives/code/decoding/milkshake_vs_h2O/images/4w_smallset_SVM_nimask.png')
# Saving the results as a Nifti file may also be important
weight_img.to_filename('/projects/niblab/bids_projects/Experiments/ChocoData/derivatives/code/decoding/milkshake_vs_h2O/images/4w_smallset_SVM_nimask.nii')

